# Zajecia 2 - Modelowanie Wieloagentowe - Sieci społeczne

## Quick intro to graphs

Sources:
* [Introduction to Graphs](http://pages.cs.wisc.edu/~paton/readings/Old/fall08/GRAPH.html)
* [A Gentle Introduction To Graph Theory](https://medium.com/basecs/a-gentle-introduction-to-graph-theory-77969829ead8)

Graph $G = (V,E)$ consists of set of **vertices** (nodes) $V$ and set of **edges** (arcs) $E$ with each edge defined as pair of adjacent vertices e.g. $e_1 \equiv (v_1,v_2)$

In **undirected** graphs order of vertices in edge definition doesn't matter.

<img src="https://miro.medium.com/max/1400/1*goT8sipQbDIoogV6Kc_3KA.jpeg" width=400p>

In **directed** graphs (**digraphs**) order of vertices in edge definition matters.

<img src="https://miro.medium.com/max/1400/1*ThD5bfLUyEx49s5S9qKKow.jpeg" width=400p>

We can keep both information about edges and vertices in **adjacency matrix**. If matrix is symmetric it means we are working with **undirected** graph.

<img src="http://pages.cs.wisc.edu/~paton/readings/Old/fall08/GRAPH-FIGURES/adMatrix.gif" width=400p>

**Facebook vs. Twitter - directed or undirected?**

On Facebook, after accepting friend request, both people are friends of one another (undirected graph). 

<img src="https://miro.medium.com/max/1400/1*qxvZX-YRBsRrmM5ePvNAQA.jpeg" width=400p>

On Twitter people we are following don't have to follow us (directed graph).

<img src="https://miro.medium.com/max/1400/1*urJTrfWn8aZdhb9A-HXZVg.jpeg" width=400p>

**Graph visualization**

<img src="http://pages.cs.wisc.edu/~paton/readings/Old/fall08/GRAPH-FIGURES/pathGraph.gif" width=100p>

Note that the layout of the graph is arbitrary. Graphs that are visualized differently may be the same graph in mathematical sense.

<img src="http://pages.cs.wisc.edu/~paton/readings/Old/fall08/GRAPH-FIGURES/arbitrary.gif" width=150p>

**Special kinds of graphs**
* A directed graph that has no cyclic paths (that contains no cycles) is called a **DAG (a Directed Acyclic Graph)**. Note: DAGs are now quite often used in Big Data processing frameworks e.g. [Spark](https://data-flair.training/blogs/dag-in-apache-spark/), [Airflow](https://airflow.apache.org/docs/stable/concepts.html).
* An undirected graph that has an edge between every pair of nodes is called a **complete graph**. A directed graph can also be a complete graph; in that case, there must be an edge from every node to every other node.
<img src="http://pages.cs.wisc.edu/~paton/readings/Old/fall08/GRAPH-FIGURES/complete.gif" width=70p>
* A graph (either directed or undirected) that has values associated with its edges is called a **weighted graph**. Edge's weight may represent road length in transportation simulations. With that setup, we can calculate shortest path between given intersections using graph traversal algorithms e.g. Dijsktra's algorithm or A* algorithm.
* A graph is connected if there is a path from every node to every other node (has only one **component**)
<img src="http://pages.cs.wisc.edu/~paton/readings/Old/fall08/GRAPH-FIGURES/connected.gif" width=500p>
* Edge can connect a node to itself (**self-loop**). Often appears in Markov Chains graph.
<img src="http://pages.cs.wisc.edu/~paton/readings/Old/fall08/GRAPH-FIGURES/selfLoop.gif" width=250p>


## Information diffusion in social graphs

Source: 

Leskovec, J., Adamic, L. A., & Huberman, B. A. (2007). The Dynamics of Viral Marketing. ACM Transactions on the Web. 
https://www.cs.cmu.edu/~jure/pubs/viral-tweb.pdf

Graph models are often used to study information diffusion in social network (e.g. viral marketing campaign) or to understand spreading of a disease in the population (epidemiology).

The classical disease propagation models are based on the stages of a disease in a host: 
* In **SIR** (susceptible/infected/recovered) model a person is first susceptible to a disease, then can become infected, and thus infectious. After the disease ceases the person is recovered or removed from network. 
* In **SIRS** or **SIS** (susceptible/infected/(recovered)/susceptible) model, recovered agent is immune for some period. The immunity can wear off, and the person becomes susceptible again.

Given a network and a set of infected nodes, the **epidemic threshold** is studied - conditions under which the disease will either dominate or die out.

However, abovementioned models are not well fitted to information spread. The problem with these types of models is that they assume a known social network over which the diseases (product recommendations) are spreading and usually a single parameter which specifies the infectiousness of the disease. This would mean that the whole population is equally susceptible to recommendations of a particular product.

Diffusion models that try to model the process of adoption of an idea or a product can generally be divided into two groups.
1. **Threshold models** where each node in the network has a threshold $t \in [0, 1]$ , typically drawn from some probability distribution. We also assign connection weights $w_{u,v}$ on the edges of the network. A node adopts the behavior if a sum of the connection weights of its neighbors that already adopted the behavior (purchased a product) is greater than the threshold $t$.
2. **Cascade models** where whenever a neighbor $v$ of node $u$ adopts, then node u also adopts with probability $p_{u,v}$. In other words, every time a neighbor of $u$ purchases a product, there is a chance that $u$ will decide to purchase as well.

## DifPy overview

Source: [Blog post about DifPy](https://john-smith-889.github.io/blog/social%20network%20analysis/difpy-diffusion-on-graphs-in-python-PL.html)

Code: https://github.com/John-smith-889/difpy

In [ ]:
# !pip install git+git://github.com/John-smith-889/difpy.git

In [ ]:
import difpy as dp

### Graph setup

In [ ]:
## graph_init() - generate by default Watts-Strogatz graph.
#Arguments:
# 1) n – no. of vertices
# 2) k – average no. of neighbors each node is connected to
# 3) rewire_prob – probability of random connection change
# 4) initiation_perc – infected/aware nodes at setup (random sampling)
# 5) show_attr – show attributes flag
# 6) draw_graph – show grap flag
#Returns:
# 1) G – NetworkX graph
# 2) pos – ndarray with information needed for plotting

G, pos = dp.graph_init(n = 10, 
                       k = 6,
                       rewire_prob = 0.1, 
                       initiation_perc = 0.1,
                       show_attr = True, 
                       draw_graph = True)

As part of the `graph_init()` function random edges' weights are drawn from exponential distribution and express probability of contact between individual agents. 

Additionally, four attributes for each node are generated:
* **receptiveness** - randomly drawn from the normal distribution and means the agent's ability to receive stimuli from the environment
* **extraversion** - generated from the normal distribution and corresponds to the level of agent extraversion in the psychological concept of Carl G. Jung's personality classification.
* **engagement** - the level of the actor's involvement in a given topic, related to his experience with the subject in which the information lies. The engagement is drawn from the exponential distribution.
* **state** - aware/unaware


Attributes are used to calculate chance of information transfer between individual nodes in the simulation module.

In [ ]:
dp.draw_graph(G, # graph
            pos, # position of nodes
            aware_color = 'red',
            not_aware_color = 'lightblue',
            legend = True)

In [ ]:
dp.graph_stats(G, 
        pos, 
        show_attr = False, 
        draw_degree = True,
        draw_graph = False)
# Clustering coeff: https://mathinsight.org/definition/clustering_coefficient
# Transitivity: https://mathinsight.org/definition/transitivity_graph

In [ ]:
import networkx as nx
import numpy as np
import copy
G_02 = nx.watts_strogatz_graph(n = 10, k = 6, p = 0.3, seed=None)
pos = nx.spring_layout(G_02)
weights = np.round(np.random.exponential(scale = 0.1, 
                size = G_02.number_of_edges()), 6).reshape(G_02.number_of_edges(),1)

# Add feature
G_02 = dp.add_feature(G_02,
                   pos,
                   feature = weights,
                   feature_type = "weights",
                   scaling = True,
                   decimals = 6,
                   show_attr = True, # show node weights and attributes
                   draw_graph = False)

In [ ]:
#Add another feature
# Create example engagement
engagement = np.round(np.random.exponential(scale = 0.1, 
size = G_02.number_of_edges()), 6).reshape(G_02.number_of_edges(),1)

# Add feature
G_02 = dp.add_feature(G_02,
                   pos,
            feature = engagement,
            feature_type = "engagement",
            scaling = True,
            show_attr = True, # show node weights and attributes
            draw_graph = False)

In [ ]:
# Add random state
dp.add_state_random(G_02, 
                    pos,
                    initiation_perc = 0.2, 
                    show_attr = True, 
                    draw_graph = True)

### Simulation

Simulation module provide functions to simulate information propagation over networks. 

It consists of the functions: 
* `simulation_step()` - calculates system state after single step of simulation
* `simulation()` - calculates system state after `n` steps
* `simulation_sequence()` - run simulation with `n` steps in many iterations (Monte Carlo model) and calculates average awareness increase

In [ ]:
G, pos = dp.graph_init(n = 10, 
                       k= 6,
                       rewire_prob = 0.1, 
                       initiation_perc = 0.1,
                       show_attr = False, 
                       draw_graph = False)

In [ ]:
# Copy graph 
G1 = copy.deepcopy(G)

In [ ]:
# Perform one simulation step
#Arguments:
# 1) G – NetworkX graph
# 2) pos – ndarray with information needed for plotting
# 3) kernel – 
## “weights” - propagation probability set by edges weights
## “WERE” - propagation based on weights-extraversion-receptiveness-engagement formula
## “custom” - propagation based on custom function
# 4) engagement_enforcement – engagement multiplier - 
##increase engagement in every iteration for agents who are aware of information
# 5) custom_kernel – custom function that control probability of information propagation between nodes -
##can operate on agents attributes
# 6) WERE_multiplier – optional, controls values  calculated with "WERE" kernel
# 7) oblivion – optional, controls agents ability to forget the information (lose awareness)
# 8) draw – show grap flag
# 9) show_attr - show attributes flag
G1 = dp.simulation_step(G1, 
                       pos,
                       kernel = 'weights', 
                       custom_kernel = None,
                       WERE_multiplier = 10, 
                       oblivion = False, 
                       engagement_enforcement = 1.01,
                       draw = True, 
                       show_attr = False)

In [ ]:
# Simulation
G2 = copy.deepcopy(G)
graph_list, avg_aware_inc_per_step = dp.simulation(G2, 
               pos,
               n = 2,                 
               kernel = 'weights', 
               custom_kernel = None,
               WERE_multiplier = 10, 
               oblivion = False, 
               engagement_enforcement = 1.01,
               draw = False, 
               show_attr = False)
    
print(avg_aware_inc_per_step)

import pprint
pprint.pprint(graph_list)

In [ ]:
G2 = copy.deepcopy(G)
avg_aware_inc_per_step = \
dp.simulation_sequence(G2, # networkX graph object
                        #pos, # p
                        n = 2, # number of steps in simulation
                        sequence_len = 1000, # sequence of simulations   
                        kernel = 'weights', # kernel type
                        custom_kernel = None, # custom kernel function
                        WERE_multiplier = 10, 
                        oblivion = False, # information oblivion feature 
                        engagement_enforcement = 1.01,
                        show_attr = False)

In [ ]:
avg_aware_inc_per_step

### Optimization

The purpose of the optimization module is to designate **n** network nodes that propagate information. The module includes `optimize_centrality()` function which calculates a set of n nodes with the highest level of closeness. The second function is `optimize_rs()`, which looks for the best vertices using the random search method.

In [ ]:
# Run function with weights as distances
#Arguments:
# 1) G – graph object
# 2) number_of_nodes – how many nodes to be returned
# 3) distance – which attribute use in optimization
# 4) wf_improved – algorithm mode switch
dp.optimize_centrality(G2,
                    distance = 'weight',
                    wf_improved = True,
                    number_of_nodes = 4)

In [ ]:
solution = dp.optimize_rs(G2,
        number_of_nodes = 2, # number of nodes to seed
        number_of_iter = 100, # number of iterations 
        log_info_interval = 10, # interval of information log 
                       
        n = 3, # number of simulation steps simulation
        sequence_len = 100, # number of simulations in one seq
                       
        kernel = 'weights', # kernel type
        custom_kernel = None, # custom kernel function
        WERE_multiplier = 10, 
        oblivion = False, # information oblivion feature 
        engagement_enforcement = 1.00
        )

### Interpretability module

Module provide tools to explore and explain attributes influence on information diffusion ability in the network.

`nodes_score_simulation()` function calculates average awareness increase in a network for every node (Monte Carlo approach)
`feature_importance()` function calculates  feature importance for every attribute assigned to agents - use ML algorithm capable of producing importance score e.g. XGBoost

In [ ]:
solution = dp.nodes_score_simulation(copy.deepcopy(G), 
                       log_info_interval = 5, # interval of information log 
                       n = 3, # number of simulation steps simulation
                       sequence_len = 100, # number of simulations in one seq
                       kernel = 'weights', # kernel type
                       custom_kernel = None, # custom kernel function
                       WERE_multiplier = 10, 
                       oblivion = False, # information oblivion feature 
                       engagement_enforcement = 1.00
                       )

In [ ]:
#Extracting attributes
X = []
for attr in list(G.nodes[0].keys())[1:]:
     X.append([G.nodes[n][attr] for n in G.nodes])
X = np.transpose(np.array(X))
X

In [ ]:
FI = dp.feature_importance(
                       G2, # NetworkX graph
                       X, # Nodes attributes 
                       show = True,
                       log_info_interval = 10, # interval of information log 
                       
                       n = 3, # number of simulation steps simulation
                       sequence_len = 20, # number of simulations in one seq
                       
                       kernel = 'weights', # kernel type
                       custom_kernel = None, # custom kernel function
                       WERE_multiplier = 10, 
                       oblivion = False, # information oblivion feature 
                       engagement_enforcement = 1.00
                       )
print(list(G.nodes[0].keys())[1:])

## Viral marketing example

Source: https://github.com/John-smith-889/difpy/blob/master/difpy_case_pl.ipynb

### Generate random network of clients

In [ ]:
G, pos = dp.graph_init(n = 300, 
                       k= 5,
                       rewire_prob = 0.2, 
                       initiation_perc = 0.00,
                       show_attr = False, 
                       draw_graph = True)

In [ ]:
dp.graph_stats(G, pos, show_attr = False, draw_degree = True,  
            draw_graph = False)

### Find "influencers"

In [ ]:
solution = dp.optimize_rs(copy.deepcopy(G),
        number_of_nodes = 5, # number of nodes to seed
        number_of_iter = 15, # number of iterations 
        log_info_interval = 5, # interval of information log 
                       
        n = 10, # number of simulation steps simulation
        sequence_len = 30, # number of simulations in one sequence
                       
        kernel = 'WERE', # kernel type
        custom_kernel = None, # custom kernel function
        WERE_multiplier = 100, 
        oblivion = False, # information oblivion feature 
        engagement_enforcement = 1.01
        )

In [ ]:
solution[1]

### Compare random campaign vs. based on influencers

#### Version 1

Goal of campaign: spread product information to 90% of all nodes ASAP

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
n = len(G.nodes)
iterations = 100

In [ ]:
steps_random = []
for _ in range(iterations):
    G_tmp = copy.deepcopy(G)
    # Make 5 random nodes aware
    dp.add_state_random(G_tmp, 
                        pos,
                        initiation_perc = 5/n, 
                        show_attr = False, 
                        draw_graph = False)
    steps = 0
    while sum([G_tmp.nodes[n]['state'] == 'aware' for n in G_tmp.nodes])/n < 0.9:
        steps +=1
        G_tmp = dp.simulation_step(G_tmp, 
                           pos,
                           kernel = 'WERE', 
                           WERE_multiplier = 100, 
                           oblivion = False, 
                           engagement_enforcement = 1.01,
                           draw = False, 
                           show_attr = False)
    steps_random.append(steps)
np.mean(steps_random)

In [ ]:
steps_infl = []
G_infl = copy.deepcopy(G)
for node in solution[1]:
    G_infl.nodes[node]['state'] = 'aware'
    
for _ in range(iterations):
    G_tmp = copy.deepcopy(G_infl)
    steps = 0
    while sum([G_tmp.nodes[n]['state'] == 'aware' for n in G_tmp.nodes])/n < 0.9:
        steps +=1
        G_tmp = dp.simulation_step(G_tmp, 
                           pos,
                           kernel = 'WERE', 
                           WERE_multiplier = 100, 
                           oblivion = False, 
                           engagement_enforcement = 1.01,
                           draw = False, 
                           show_attr = False)
    steps_infl.append(steps)
np.mean(steps_infl)

In [ ]:
sns.distplot(steps_random , color="green", label="Random spreaders")
sns.distplot(steps_infl , color="red", label="Influencers")
plt.legend();

#### Version 2

Goal of campaign: check spread among network after 5 time steps

In [ ]:
k = 5

In [ ]:
aware_random = []
for _ in range(iterations):
    G_tmp = copy.deepcopy(G)
    # Make 5 random nodes aware
    dp.add_state_random(G_tmp, 
                        pos,
                        initiation_perc = 5/n, 
                        show_attr = False, 
                        draw_graph = False)
    for _ in range(k):
        G_tmp = dp.simulation_step(G_tmp, 
                           pos,
                           kernel = 'WERE', 
                           WERE_multiplier = 100, 
                           oblivion = False, 
                           engagement_enforcement = 1.01,
                           draw = False, 
                           show_attr = False)
    aware_random.append(sum([G_tmp.nodes[n]['state'] == 'aware' for n in G_tmp.nodes]))
np.mean(aware_random)

In [ ]:
aware_infl = []
G_infl = copy.deepcopy(G)
for node in solution[1]:
    G_infl.nodes[node]['state'] = 'aware'
    
for _ in range(iterations):
    G_tmp = copy.deepcopy(G_infl)
    for _ in range(k):
        G_tmp = dp.simulation_step(G_tmp, 
                           pos,
                           kernel = 'WERE',
                           WERE_multiplier = 100, 
                           oblivion = False, 
                           engagement_enforcement = 1.01,
                           draw = False, 
                           show_attr = False)
    aware_infl.append(sum([G_tmp.nodes[n]['state'] == 'aware' for n in G_tmp.nodes]))
np.mean(aware_infl)

In [ ]:
sns.distplot(aware_random , color="blue", label="Random spreaders")
sns.distplot(aware_infl , color="red", label="Influencers")
plt.legend();